In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-driver
%pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadc

In [ ]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
import google_colab_selenium as gs
import json
import re

# Pittsburgh events calendar

In [ ]:
def find_soup(url):
  # Set up Selenium's browser driver
  driver = gs.Chrome()

  # Open the target webpage
  driver.get(url)  # Replace with the actual URL

  # Loop to click the "Show More" button until it can no longer be clicked
  while True:
      try:
          # Find the "Show More" button
          show_more_button = driver.find_element(By.CSS_SELECTOR, "div.ldm span.textnode")

          # Check if the button is visible and click it
          if show_more_button.is_displayed():
              print("Clicking 'Show More' button...")
              driver.execute_script("arguments[0].click();", show_more_button)
              time.sleep(2)  # Wait for content to load
          else:
              break  # Exit the loop if the button is not visible
      except Exception as e:
          print("No more 'Show More' button to click.")
          break

  # Once the page is fully loaded, use BeautifulSoup to parse the content
  soup = BeautifulSoup(driver.page_source, 'html.parser')

  # Close the browser
  driver.quit()
  return soup

In [ ]:
def store_file(soup, filename, return_soup=False):
    event_items = soup.find('ul', {'class': 'dates-list'}).find_all('li', {'class': 'date-row'}) if soup.find('ul', {'class': 'dates-list'}) else None

    events = []

    if event_items:
        for item in event_items:
            event_date = item.find('div', {'class': 'date'}).text.strip() if item.find('div', {'class': 'date'}) else 'N/A'
            event_time = item.find('div', {'class': 'time'}).text.strip() if item.find('div', {'class': 'time'}) else 'N/A'

            event_date = re.sub(r'\s+', ' ', event_date)
            if event_date[0] == 'O' and event_date < 'Oct 27 2024':
                continue
            event_time = f"{event_date}|{event_time}"
            event_time = re.sub(r'\s+', ' ', event_time)

            event_name = item.find('div', {'class': 'venue'}).find('a').text.strip() if item.find('div', {'class': 'venue'}).find('a') else 'N/A'

            destination_div = item.find('div', {'class': 'date-desc'})
            destination = destination_div.find('a').text.strip() if destination_div.find('a') else 'N/A'

            location = item.find('span', {'class': 'location'}).text.strip() if item.find('span', {'class': 'location'}) else 'N/A'

            price = item.find('div', {'class': 'from-price'}).text.strip() if item.find('div', {'class': 'from-price'}) else 'N/A'

            event = {
                'event_name': event_name,
                'event_time': event_time,
                'location': f"{destination}|{location}",
                'price': price
            }
            events.append(event)
    else:
        print("No 'dates-list' found.")

    if return_soup:
        return events

    with open(filename, 'w') as f:
        json.dump(events, f, indent=4)

    print(f"Data saved to {filename}")

In [ ]:
from pickle import TRUE
urls=['https://pittsburgh.events/october/','https://pittsburgh.events/november/','https://pittsburgh.events/december/']
files=['october.json','november.json','december.json']

events=[]
for i in range(len(urls)):
  soup=find_soup(urls[i])
  events.extend(store_file(soup,files[i],return_soup=TRUE))
with open('/content/drive/MyDrive/11711/assignment2/data/PittsburgEventCalendar.json', 'w') as f:
    json.dump(events, f, indent=4)

<IPython.core.display.Javascript object>

Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...



<IPython.core.display.Javascript object>

Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...



<IPython.core.display.Javascript object>

Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...



In [ ]:
soup=find_soup('https://pittsburgh.events/october/')
# 假设你已经获得了 soup 对象
# 找到包含日期列表的元素
date_list = soup.find('ul', {'class': 'dates-list'})

# 如果找到 date-list，提取其中每个 'date-row' 的信息
if date_list:
    date_rows = date_list.find_all('li', {'class': 'date-row'})

    # 遍历每个日期条目并提取信息
    for row in date_rows:
        date = row.find('div', {'class': 'date'}).text.strip() if row.find('div', {'class': 'date'}) else 'N/A'
        # 输出每个事件的信息
        print(f"Date: {date}")
else:
    print("No 'dates-list' found.")

<IPython.core.display.Javascript object>

Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...
Clicking 'Show More' button...


KeyboardInterrupt: 

#Downtown Pittsburgh events calendar

In [ ]:
with open('f.txt', 'r') as f:
    text = [line.strip() for line in f]

In [ ]:
events=[]
line=0
while line<len(text):
  if text[line]=='':
    line+=1
    continue
  event = {
      'event_name': text[line],
      'event_time': text[line+1],
  }
  if text[line+2]=='Virtual Event':
    event['location']='Virtual Event'
    content_line=line+3
  elif text[line+2]=='August Wilson African American Cultural Center':
      event['location']=text[line+2]
      content_line=line+3
  else:
      event['location']=text[line+2]+'|'+text[line+3]+','+text[line+4]
      content_line=line+5
  if text[content_line]!='':
    event['content']=[]
  while text[content_line]!='':
    event['content'].append(text[content_line])
    content_line+=1
    if content_line==len(text) or text[content_line]=='':
      break
  events.append(event)
  del event
  line=content_line
print(events)

[{'event_name': 'Mod Sun & lovelytheband: Here’s Your Flowers Tour', 'event_time': 'Oct 27, 2024 - Oct 27, 2024 | 6:00 pm - 10:00 pm', 'location': 'Stage AE|400 N Shore Dr,Pittsburgh, PA 15212'}, {'event_name': 'Lucky Draw', 'event_time': 'Oct 27, 2024 - Oct 27, 2024 | 7:30 pm - 9:30 pm', 'location': 'Arcade Comedy Theater|943 Liberty Ave,Pittsburgh, PA 15222', 'content': ['Arcade Comedy Theater presents “Lucky Draw” an improv comedy show where half of the cast is drawn at random! Each show, Arcade Academy students and team members drop their names into box at the start of the night. Throughout the evening, the hosts will be playing short form games and doing a long-form set that mixes up these performers alongside special guests, teachers, and coaches from the Arcade community. If you’re a student or improviser, this is a perfect chance to maybe try out new games or play with some seasoned performers. And if you’re just looking for a fun, unique night out Ñ you’ll get a chance to see 

In [ ]:
events

[{'event_name': 'Mod Sun & lovelytheband: Here’s Your Flowers Tour',
  'event_time': 'Oct 27, 2024 - Oct 27, 2024 | 6:00 pm - 10:00 pm',
  'location': 'Stage AE|400 N Shore Dr,Pittsburgh, PA 15212'},
 {'event_name': 'Lucky Draw',
  'event_time': 'Oct 27, 2024 - Oct 27, 2024 | 7:30 pm - 9:30 pm',
  'location': 'Arcade Comedy Theater|943 Liberty Ave,Pittsburgh, PA 15222',
  'content': ['Arcade Comedy Theater presents “Lucky Draw” an improv comedy show where half of the cast is drawn at random! Each show, Arcade Academy students and team members drop their names into box at the start of the night. Throughout the evening, the hosts will be playing short form games and doing a long-form set that mixes up these performers alongside special guests, teachers, and coaches from the Arcade community. If you’re a student or improviser, this is a perfect chance to maybe try out new games or play with some seasoned performers. And if you’re just looking for a fun, unique night out Ñ you’ll get a cha

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/DowntownPittsburghEventsCalendar.json', 'w') as f:
    json.dump(events, f, indent=4)

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/DowntownPittsburghEventsCalendar.json', 'r') as f:
    events = json.load(f)
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.txt', 'r') as f:
    text = [line.strip() for line in f]
line=0
while line<len(text):
  if text[line]=='':
    line+=1
    continue
  event = {
      'event_name': text[line],
      'event_time': text[line+1],
  }

  content_line=line+2

  if text[content_line]!='':
    event['content']=[]
  while text[content_line]!='':
    event['content'].append(text[content_line])
    content_line+=1
    if content_line==len(text) or text[content_line]=='':
      break
  events.append(event)
  del event
  line=content_line
with open('/content/drive/MyDrive/11711/assignment2/data/DowntownPittsburghEventsCalendar.json', 'w') as f:
    json.dump(events, f, indent=4)

#Pittsburgh City Paper events

In [ ]:
urls=['https://www.pghcitypaper.com/pittsburgh/EventSearch?keywords=annual&sortType=date&v=d','https://www.pghcitypaper.com/pittsburgh/EventSearch?narrowByDate=2024-10-27-to-2024-12-31&sortType=date&v=d']
p1='https://www.pghcitypaper.com/pittsburgh/EventSearch?narrowByDate=2024-10-27-to-2024-12-31&page='
p2='&sortType=date&v=d'
for i in range(2,18):
  urls.append(p1+str(i)+p2)

In [ ]:
from posixpath import split
def store_file(soup, filename,return_soup=False):
    event_items = soup.find_all('div', {'class': "fdn-pres-content uk-flex-1 uk-padding-xsmall-right fdn-event-search-text-block"})

    events = []

    if event_items:
        for item in event_items:
            event_name = item.find('p', {'class': 'fdn-teaser-headline'}).text.strip() if item.find('p', {'class': 'fdn-teaser-headline'}) else 'N/A'

            event_link = item.find('a')['href'] if item.find('a') else 'N/A'

            event_time = item.find('p', {'class': 'fdn-teaser-subheadline'}).text.strip() if item.find('p', {'class': 'fdn-teaser-subheadline'}) else 'N/A'

            location = item.find('p', {'class': 'fdn-event-teaser-location'}).text.strip() if item.find('p', {'class': 'fdn-event-teaser-location'}) else 'N/A'

            price = item.find('span', {'class': 'uk-margin-xsmall-top'}).text.strip() if item.find('span', {'class': 'uk-margin-xsmall-top'}) else 'N/A'

            content = item.find('div', {'class': 'fdn-teaser-description'}).text.strip() if item.find('div', {'class': 'fdn-teaser-description'}) else 'N/A'

            event = {
                'event_name': event_name,
                'event_time': event_time,
                'location': location,
                'price': price,
                'content': content
            }

            events.append(event)
    else:
        print("No event items found.")

    if return_soup:
        return events

    with open(filename, 'w') as f:
        json.dump(events, f, indent=4)

    print(f"Data saved to {filename}")

In [ ]:
def find_soup(url):
  # Set up Selenium's browser driver
  driver = gs.Chrome()

  # Open the target webpage
  driver.get(url)  # Replace with the actual URL

  # Once the page is fully loaded, use BeautifulSoup to parse the content
  soup = BeautifulSoup(driver.page_source, 'html.parser')

  # Close the browser
  driver.quit()
  return soup

In [ ]:
soup=find_soup(urls[0])
soup=store_file(soup,'a',return_soup=True)


<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/tem.json', 'w') as f:
    json.dump(soup, f, indent=4)

In [ ]:
from pickle import TRUE

events=[]
for i in range(len(urls)):
  soup=find_soup(urls[i])
  events.extend(store_file(soup,files[0],return_soup=TRUE))
with open('/content/drive/MyDrive/11711/assignment2/data/PittsburghCityPaperEvents.json', 'w') as f:
    json.dump(events, f, indent=4)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#CampusEventsPage

In [ ]:
events_data = [
    {"event_time": "October 27, 2024", "location": "Honolulu, HI", "event_name": "HI - CMU Meetup: Rock Climbing"},
    {"event_time": "November 01, 2024", "location": "Pittsburgh, PA", "event_name": "Homecoming 2024"},
    {"event_time": "November 01, 2024", "location": "Pittsburgh, PA", "event_name": "Alumni Awards Ceremony"},
    {"event_time": "November 02, 2024", "location": "New York, NY", "event_name": "NYC - Fall Happy Hour"},
    {"event_time": "November 06, 2024", "location": "Atlanta, GA", "event_name": "ATL - Alumni Event Planning Meeting"},
    {"event_time": "November 09, 2024", "location": "Pittsburgh, PA", "event_name": "PIT - Saint Anthony Chapel Tour"},
    {"event_time": "November 09, 2024", "location": "Marietta, GA", "event_name": "ATL- Hands-on Workshop to Build Your LinkedIn Brand"},
    {"event_time": "November 11, 2024", "location": "Webinar, Online", "event_name": "PGH - November Network Planning Meeting"},
    {"event_time": "November 11, 2024", "location": "Decatur, GA", "event_name": "ATL- Scotch Tasting in Decatur"},
    {"event_time": "November 12, 2024", "location": "New York, NY", "event_name": "NYC - Art Collecting With Confidence"},
    {"event_time": "November 13, 2024", "location": "La Jolla, CA", "event_name": "SD - Wine Tasting Social"},
    {"event_time": "November 16, 2024", "location": "New York, NY", "event_name": "HLAA - Coffee with Glenn and Jonathan: Thanksgiving Edition"},
    {"event_time": "December 13, 2024", "location": "Elmhurst, IL", "event_name": "CHI - West Burbs Social Lunch at Gia Mia"},
    {"event_time": "December 19, 2024", "location": "Sandy Springs, GA", "event_name": "ATL - Vienna Boys Choir: Christmas in Vienna"},
    {"event_time": "December 22, 2024", "location": "Honolulu, HI", "event_name": "HI - Network Holiday Party"}
]

# Saving to a JSON file
filename = "/content/drive/MyDrive/11711/assignment2/data/CampusEventsPage.json"
with open(filename, 'w') as f:
    json.dump(events_data, f, indent=4)

# CMU events calendar

In [ ]:
urls=[]
for i in range(27,32):
  urls.append('https://events.cmu.edu/day/date/202410'+str(i))
for i in range(1,31):
  urls.append('https://events.cmu.edu/day/date/202411'+str(i))
for i in range(1,32):
  urls.append('https://events.cmu.edu/day/date/202412'+str(i))

In [ ]:
def find_soup(url):
  # Set up Selenium's browser driver
  driver = gs.Chrome()
  driver.get(url)
  # Wait for the lw_cal_event_list to be rendered on the page
  try:
      WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.CLASS_NAME, "lw_cal_event_list"))
      )
  except Exception as e:
      print(f"Error: {e}")

  # Get the page source once the desired element is present
  soup = BeautifulSoup(driver.page_source, 'html.parser')

  # Close the driver
  driver.close()

  # Extract the desired element
  events_list = soup.find('div', {'class': 'lw_cal_event_list'})

  # Print or return the extracted content
  print(events_list)
  return events_list

In [ ]:
soup=find_soup(urls[0])

<IPython.core.display.Javascript object>

<div class="lw_cal_event_list"> <div class="lw_cal_event lw_has_image lw_category_open_to_the_public lw_category_athletics_recreation lw_category_alumni lw_category_faculty lw_category_general_public lw_category_graduate_students lw_category_staff lw_category_undergraduate_students lw_category_youth_(k-12)" data-id="11977"> <div class="lw_cal_event_info"> <div class="lw_events_location"> </div> <div class="lw_events_time"> <span class="lw_start_time">Noon</span> - <span class="lw_end_time">3 p.m.</span> </div> <span class="lw_item_thumb"> <a href="/event/11977-womens-volleyball-case-western-reserve-vs-carnegie"><picture class="lw_image">
<source data-origin="responsive" srcset="https://events.cmu.edu/live/image/scale/2x/gid/4/width/200/height/200/crop/1/src_region/0,0,400,400/27_cmu-athletics.rev.1717011793.webp 2x" type="image/webp"/>
<source data-origin="responsive" srcset="https://events.cmu.edu/live/image/scale/2x/gid/4/width/200/height/200/crop/1/src_region/0,0,400,400/27_cmu-athl

In [ ]:
soup.find_all('div', {'id':"main-content"})

[<div class="col-lg-9" id="main-content">
 <div class="clearfix" id="lw_cal_view_selector"> <ul role="navigation"> <li class="today"><a aria-label="October 15, 2024 Events View" href="#!view/day/date/20241015">October 15, 2024</a></li> <li class="day"><a aria-label="Day View" href="#!view/day">Day</a></li> <li class="week"><a aria-label="Week View" href="#!view/week">Week</a></li> <li class="month"><a aria-label="Month View" href="#!view/month">Month</a></li> <li class="all"> <a aria-label="List All Upcoming Events" href="#!view/all"><span class="m">List </span><span>All Upcoming Events</span></a> <span class="lw_hidden" id="lw_cal_events_badge"></span> </li> </ul> </div>
 <div id="lw_cal">
 <div id="lw_cal_header"> <div id="lw_cal_date_selector"></div> <div id="lw_cal_scroll_links"></div> <h5 id="lw_cal_this_day"></h5> <div id="lw_cal_showing"></div> </div>
 <div class="lw_clearfix" id="lw_cal_body">
 <div class="lw_cal_loading" id="lw_cal_events" tabindex="-1">
 </div>
 </div>
 <div 

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
import json

def store_file(url, filename, return_soup=False):
    # Set up Selenium's browser driver
    driver = gs.Chrome()  # Ensure this is correctly configured for your setup
    driver.get(url)

    # Wait for the lw_cal_event_list to be rendered on the page
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "lw_cal_event_list"))
        )
    except Exception as e:
        print(f"Error: {e}")

    # Get the page source after waiting
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Close the driver after getting the page source
    driver.close()

    # Find the main event list container
    month=soup.find('span', class_="lw_cal_today_month")
    day=soup.find('span', class_="lw_cal_today_day")
    event_list_container = soup.find('div', class_="lw_cal_event_list")

    events = []

    if event_list_container:
        # Find all individual events
        event_items = event_list_container.find_all('div', class_="lw_cal_event")

        # Iterate over each event and extract details
        for item in event_items:
            event = {}
            # Get event name from the title section
            event_name = item.find('div', {'class': 'lw_events_title'}).text.strip() if item.find('div', {'class': 'lw_events_title'}) else 'N/A'
            event['event_name']=event_name

            if item.find('div', {'class': 'lw_events_time'}):
              # Get event time
              event_time = item.find('div', {'class': 'lw_events_time'}).text.strip()
              event['event_time']=month.text+' '+day.text+',2024'+'|'+event_time

            if item.find('div', {'class': 'lw_events_location'}):
              location = item.find('div', {'class': 'lw_events_location'}).text.strip()
              if location !='':
                event['location']=location

            if item.find('div', {'class': 'lw_events_summary'}):
              content = item.find('div', {'class': 'lw_events_summary'}).text.strip()
              event['content']=content

            # Append the event to the events list
            events.append(event)
    else:
        print("No event list container found.")

    if return_soup:
        return events

    # Save events to a JSON file
    with open(filename, 'w') as f:
        json.dump(events, f, indent=4)

    print(f"Data saved to {filename}")


In [ ]:
print(urls[0])
store_file(urls[0], 'a',return_soup=True)

https://events.cmu.edu/day/date/20241027


<IPython.core.display.Javascript object>

[{'event_name': '(Women’s Volleyball) Case Western Reserve vs. Carnegie Mellon',
  'event_time': 'Noon - 3 p.m.|Oct 27,2024'},
 {'event_name': '(Women’s Volleyball) WashU vs. Carnegie Mellon',
  'event_time': '2 - 5 p.m.|Oct 27,2024'},
 {'event_name': 'Voter Registration Assistance',
  'event_time': '4 - 7 p.m. Weekly (to Nov 6)|Oct 27,2024',
  'location': 'Nexus of Civic Engagement, Room 102, Cohon Univers...',
  'content': 'A member of the CMU Votes team will be available to help you register to vote, check your voter registration status, and/or find your polling place for the upcoming election'},
 {'event_name': 'Here to Stay: Celebrating 40 Years at the Architecture Archives',
  'event_time': 'All Day|Oct 27,2024',
  'location': 'Hunt Library, Fourth Floor',
  'content': 'This University Libraries’ exhibition celebrates the 40th anniversary of the Architecture Archives, which was founded to create, conserve and promote the rich history of Pittsburgh’s buildings and tell part of the

In [ ]:
events=[]
for i in range(len(urls)):
  events.extend(store_file(urls[i],'',return_soup=True))
with open('/content/drive/MyDrive/11711/assignment2/data/CMUeventsCalendar.json', 'w') as f:
    json.dump(events, f, indent=4)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# utilize for sports (greedy way)

In [103]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


def get_subpage_links(unwanted_keywords,area,url, depth, current_depth=0, visited=None):
    if visited is None:
        visited = set()  # Keep track of visited URLs to avoid duplicates

    # Stop recursion if the maximum depth is reached
    if current_depth > depth:
        return []

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return []


    # Find all <a> tags and filter out unwanted links
    links = []
    for a_tag in soup.find_all('a', href=True):
        link = a_tag['href']

        # Convert relative links to absolute
        link = urljoin(url, link)

        # Only include links containing the word "pirates"
        if area in link.lower() and not any(keyword in link.lower() for keyword in unwanted_keywords) and link not in visited:
            visited.add(link)  # Mark the link as visited
            links.append(link)

            # Recursively fetch sublinks if not too deep
            if current_depth < depth:
                sublinks = get_subpage_links(unwanted_keywords,area,link, depth, current_depth + 1, visited)
                links.extend(sublinks)

    return links

# # Example usage
# url = "https://www.mlb.com/pirates"  # Replace with the base URL
# depth = 5  # Set the desired depth for recursion
# subpage_links = get_subpage_links(url, depth)

# print(f"Found {len(subpage_links)} subpage links (filtered and recursive):")
# for link in subpage_links:
#     print(link)



In [60]:
# # Example usage
# url = "https://www.mlb.com/pirates/ballpark/tours"  # Replace with the website you want to scrape
# structured_text = scrape_structured_text(url)


# if structured_text:
#     # Optionally save the result as JSON for easier handling
#     import json
#     with open("structured_text.json", "w", encoding="utf-8") as f:
#         json.dump(structured_text, f, ensure_ascii=False, indent=4)
#     print("Structured text saved to 'structured_text.json'")
# else:
#     print("Failed to scrape content.")

Structured text saved to 'structured_text.json'


In [84]:
# # Example usage
# url = "https://www.steelers.com/"  # Replace with the base URL
# depth = 6  # Set the desired depth for recursion
# subpage_links = get_subpage_links('steelers',url, depth)

# print(f"Found {len(subpage_links)} subpage links (filtered and recursive):")
# for link in subpage_links:
#     print(link)

Found 120 subpage links (filtered and recursive):
https://www.steelers.com/#main-content
https://www.steelers.com/
https://www.steelers.com
https://www.steelers.com#main-content
https://www.steelers.com/game-day/2024/week7/jets-at-steelers/
https://www.steelers.com/game-day/2024/week7/jets-at-steelers/#main-content
https://www.steelers.com/schedule/index
https://www.steelers.com/team/players-roster/
https://www.steelers.com/fans/
https://www.steelers.com/game-day/2024/week7/jets-at-steelers/#2ndlevel
https://www.steelers.com/museum/
https://www.steelers.com/audio/steelers-nation-radio/
https://www.steelers.com/audio/podcast/
https://www.steelers.com/history/
https://www.steelers.com/legends/
https://www.steelers.com/women/
https://www.steelers.com/youth/
https://www.steelers.com/ireland/
https://www.steelers.com/deutschland/
https://www.steelers.com/community/index
https://www.steelers.com/tickets
https://www.steelers.com/tickets/index
https://www.steelers.com/game-day/2024/week7/jets-

In [90]:
import requests
from bs4 import BeautifulSoup

# Function to recursively extract structured text and ignore unwanted tags
def extract_text_with_structure(element):
    # Define tags that should be excluded (e.g., script, style, etc.)
    exclude_tags = ['script', 'style', 'meta', 'link', 'noscript']

    # If the element is in exclude tags, skip it
    if element.name in exclude_tags:
        return None

    # If the element contains only text, return the text
    if element.string:
        return element.string.strip()

    # Otherwise, create a list to store the structured data
    structured_data = []

    # Iterate over the children of the current element
    for child in element.children:
        if child.name:  # Check if the child is an HTML tag
            child_data = extract_text_with_structure(child)
            if child_data:  # Only add if there is text
                structured_data.append(child_data)
        elif child.string and child.string.strip():  # If the child is a text node
            structured_data.append(child.string.strip())

    # If the structured data list has only one item, return it directly
    if len(structured_data) == 1:
        return structured_data[0]

    # Return the structured data as a list
    return structured_data

# Function to scrape and return structured text
def scrape_structured_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

    # Remove unwanted tags such as <script>, <style>, etc.
    for tag in soup(['script', 'style', 'meta', 'link', 'noscript']):
        tag.decompose()  # This removes the tag and its content

    # Extract text from the body of the webpage
    body = soup.body
    if body:
        structured_text = extract_text_with_structure(body)
    else:
        return None

    return structured_text




In [91]:
urls=['https://www.mlb.com/pirates','https://www.nhl.com/penguins','https://www.steelers.com/']
areas=['pirates','penguins','steelers']
unwanted_keywords = ['snapchat','dubliner','shop','news', 'scores', 'recent', 'stats', 'states','thepiratespress','issuu','video','photos','espanol','itunes','google','twitter','instagram','youtube','tiktok','facebook']

for i in range(len(urls)):
  subpage_links = get_subpage_links(unwanted_keywords,areas[i],urls[i], 6)
  print(f"Found {len(subpage_links)} subpage links (filtered and recursive):")
  data=[]
  for link in subpage_links:
    structured_text = scrape_structured_text(link)
    if structured_text:
      data.extend(structured_text)
  with open('/content/drive/MyDrive/11711/assignment2/data/greedy_'+areas[i]+'.json', 'w') as f:
    json.dump(data, f, indent=4)

Found 100 subpage links (filtered and recursive):


Found 266 subpage links (filtered and recursive):
Error fetching https://penguinspride.com: HTTPSConnectionPool(host='penguinspride.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)')))
Error fetching mailto:info@pittsburghpenguinsfoundation.org: No connection adapters were found for 'mailto:info@pittsburghpenguinsfoundation.org'


Found 120 subpage links (filtered and recursive):


In [96]:
urls=['https://www.mlb.com/pirates','https://www.nhl.com/penguins','https://www.steelers.com/']
areas=['pirates','penguins','steelers']

def clean_data(data, unwanted_strings):
    if isinstance(data, list):
        # Iterate through each item in the list and clean it recursively
        return [clean_data(item, unwanted_strings) for item in data if clean_data(item, unwanted_strings) != ""]
    elif isinstance(data, str):
        # If the data is a string, check if it matches any of the unwanted strings
        if data in unwanted_strings:
            return ""  # Return an empty string if it's an unwanted string
        return data  # Return the string if it's not unwanted
    else:
        return data  # If it's not a list or string, return the data as is

for i in range(len(urls)):
  with open('/content/drive/MyDrive/11711/assignment2/data/greedy_'+areas[i]+'.json', 'r') as f:
    data=json.load(f)
  with open('/content/drive/MyDrive/11711/assignment2/data/greedy_'+areas[i]+'.json', 'w') as f:
    json.dump(clean_data(data, ["[if IE 9]><video style=\"display: none;\"><![endif]","[if IE 9]><video style=\"display: none; \"><![endif]",
            "[if IE 9]></video><![endif]"]), f, indent=4)

In [ ]:
urls=['https://www.thefrickpittsburgh.org']
areas=['thefrickpittsburgh']
unwanted_keywords=[]
for i in range(len(urls)):
  subpage_links = get_subpage_links(unwanted_keywords,areas[i],urls[i], 11)
  print(f"Found {len(subpage_links)} subpage links (filtered and recursive):")
  data=[]
  for link in subpage_links:
    structured_text = scrape_structured_text(link)
    if structured_text:
      data.extend(structured_text)
  with open('/content/drive/MyDrive/11711/assignment2/data/greedy_'+areas[i]+'.json', 'w') as f:
    json.dump(clean_data(data, ["[if IE 9]><video style=\"display: none;\"><![endif]","[if IE 9]><video style=\"display: none; \"><![endif]",
            "[if IE 9]></video><![endif]"]), f, indent=4)

Error fetching mailto:mrolla@thefrickpittsburgh.org?subject=Why%20is%20the%20Car%20and%20Carriage%20Museum%20closed%20and%20what's%20next%3F: No connection adapters were found for "mailto:mrolla@thefrickpittsburgh.org?subject=Why%20is%20the%20Car%20and%20Carriage%20Museum%20closed%20and%20what's%20next%3F"
Error fetching mailto:kcady@thefrickpittsburgh.org: No connection adapters were found for 'mailto:kcady@thefrickpittsburgh.org'


Found 301 subpage links (filtered and recursive):


Error fetching mailto:info@thefrickpittsburgh.org: No connection adapters were found for 'mailto:info@thefrickpittsburgh.org'


In [99]:
def get_subpage_links_general(url, depth, current_depth=0, visited=None):
    if visited is None:
        visited = set()  # Keep track of visited URLs to avoid duplicates

    # Stop recursion if the maximum depth is reached
    if current_depth > depth:
        return []

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return []

    # Keywords to filter out and include
    unwanted_keywords = ['news', 'scores', 'recent', 'stats', 'states']

    # Find all <a> tags and filter out unwanted links
    links = []
    for a_tag in soup.find_all('a', href=True):
        link = a_tag['href']

        # Convert relative links to absolute
        link = urljoin(url, link)

        # Only include links containing the word "pirates"
        if 'visitpittsburgh' in link.lower() and ('sports' in link.lower() or 'pirates' in link.lower() or 'steelers' in link.lower() or 'penguins' in link.lower())and not any(keyword in link.lower() for keyword in unwanted_keywords) and link not in visited:
            visited.add(link)  # Mark the link as visited
            links.append(link)

            # Recursively fetch sublinks if not too deep
            if current_depth < depth:
                sublinks =  get_subpage_links_general(link, depth, current_depth + 1, visited)
                links.extend(sublinks)

    return links

In [100]:
# Example usage
url = "https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams"  # Replace with the base URL
depth = 6  # Set the desired depth for recursion
subpage_links = get_subpage_links_general(url, depth)

print(f"Found {len(subpage_links)} subpage links (filtered and recursive):")
for link in subpage_links:
    print(link)

Found 31 subpage links (filtered and recursive):
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams#main
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/#main
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/pittsburgh-steelers/
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/pittsburgh-steelers/#main
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/terrible-towel/
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/terrible-towel/#main
https://www.visitpittsburgh.com/blog/guide-to-acrisure-stadium-for-steelers-fans/
https://www.visitpittsburgh.com/directory/pittsburgh-steelers-event-meeting-venues/
https://www.visitpittsburgh.com/blog/steelers-nation-lets-bring-it-in/
https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/college-sports/
https://www.visitpittsburgh.com/things-to-do/arts-

In [102]:
urls=['https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams']
areas=['sportsGeneral']
for i in range(len(urls)):
  subpage_links = get_subpage_links_general(urls[i], 6)
  print(f"Found {len(subpage_links)} subpage links (filtered and recursive):")
  data=[]
  for link in subpage_links:
    structured_text = scrape_structured_text(link)
    if structured_text:
      data.extend(structured_text)
  with open('/content/drive/MyDrive/11711/assignment2/data/greedy_'+areas[i]+'.json', 'w') as f:
    json.dump(clean_data(data, ["[if IE 9]><video style=\"display: none;\"><![endif]","[if IE 9]><video style=\"display: none; \"><![endif]",
            "[if IE 9]></video><![endif]"]), f, indent=4)

Found 31 subpage links (filtered and recursive):


#pirates

In [ ]:
import requests
from bs4 import BeautifulSoup
# 发送HTTP请求并获取HTML内容
url = 'https://en.wikipedia.org/wiki/Pittsburgh_Pirates'
response = requests.get(url)
html = response.text
# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(html, 'html.parser')
# 提取所需数据
data = []
for tag in soup.find_all('div', {'class': 'mw-content-ltr'}):
  for p in tag.find_all('p'):
    text = p.get_text()
    if text:
        data.append(text)
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.json', 'r') as f:
    data = json.load(f)

In [ ]:
data['pirates timeline/history']={}
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.txt', 'r') as f:
    text = f.read()
    text = text.split('\n')
data['pirates timeline/history']["The early years"]=text
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.json', 'r') as f:
    data = json.load(f)
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.txt', 'r') as f:
    text = f.read()
    text = text.split('\n')
data['pirates timeline/history']["THE PNC PARK YEARS"]=text
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.json', 'w') as f:
    json.dump(data, f, indent=4)

# steelers

In [ ]:
import requests
from bs4 import BeautifulSoup
# 发送HTTP请求并获取HTML内容
url = 'https://en.wikipedia.org/wiki/Pittsburgh_Steelers'
response = requests.get(url)
html = response.text
# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(html, 'html.parser')
# 提取所需数据
data = []
for tag in soup.find_all('div', {'class': 'mw-content-ltr'}):
  for p in tag.find_all('p'):
    text = p.get_text()
    if text:
        data.append(text)
with open('/content/drive/MyDrive/11711/assignment2/data/Steelers.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/steelers.json', 'r') as f:
    data = json.load(f)
with open('/content/drive/MyDrive/11711/assignment2/data/pirates.txt', 'r') as f:
    text = f.read()
    text = text.split('\n')
    text = [line for line in text if line.strip() != '']
data['steelers history']["HALL OF HONOR MUSEUM"]=text
with open('/content/drive/MyDrive/11711/assignment2/data/steelers.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/steelers.json', 'r') as f:
    data = json.load(f)
for i in range(len(data['steelers history']["HALL OF HONOR MUSEUM"])):
  data['steelers history']["HALL OF HONOR MUSEUM"][i]=data['steelers history']["HALL OF HONOR MUSEUM"][i].replace('\t',' ')
for i in range(len(data['steelers history']["Steelers Hall of Honor Member"])):
  data['steelers history']["Steelers Hall of Honor Member"][i]=data['steelers history']["Steelers Hall of Honor Member"][i].replace('\t',' ')
with open('/content/drive/MyDrive/11711/assignment2/data/steelers.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

url = 'https://www.steelers.com/team/players-roster/'
response = requests.get(url)
html = response.text
# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(html, 'html.parser')

# 找到表格元素
table = soup.find("table")

with open("data.csv", "w", newline="", encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    # Write header (optional if you don't have <th> elements)
    header = [th.text.strip() for th in table.find_all("th")]
    if header:
        writer.writerow(header)

    # Loop through table rows <tr>
    for row in table.find_all("tr"):
        # Extract all <td> elements
        data = [td.text.strip() for td in row.find_all("td")]

        # Only write rows with data (ignore rows without <td> elements)
        if data:
            writer.writerow(data)

In [ ]:
data

['\n\n\n\n\n\n\nRussell Wilson\n\n',
 '3',
 'QB',
 '5-11',
 '206',
 '35',
 '13',
 'Wisconsin']

#penguins

In [ ]:
import pandas as pd

# Load the Excel file
excel_file = '/content/drive/MyDrive/11711/assignment2/data/penguins.xlsx'  # Replace with your file path

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file)

# Convert the DataFrame to a CSV file
csv_file = '/content/drive/MyDrive/11711/assignment2/data/penguins_teams_goalies_players.csv'  # Replace with your desired output file name
df.to_csv(csv_file, index=False)

print(f'Excel file has been converted to {csv_file}')

Excel file has been converted to /content/drive/MyDrive/11711/assignment2/data/penguins_teams_goalies_players.csv


In [ ]:
!apt update


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
39 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

driver = gs.Chrome()

# Open the target webpage
driver.get('https://www.nhl.com/penguins/roster')  # Replace with the actual URL

# Once the page is fully loaded, use BeautifulSoup to parse the content
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 找到表格元素
table = soup.find("table")

# 创建CSV文件并写入表头
with open("data.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    header = [th.text for th in table.find_all("th")]
    writer.writerow(header)

    # 遍历表格行
    for row in table.find_all("tr"):
        # 提取每行的数据
        data = [td.text for td in row.find_all("td")]
        writer.writerow(data)

<IPython.core.display.Javascript object>

GoogleColabSeleniumError: Failed to update and upgrade APT
	
        If you believe this isn't due to a configuration error on your end, please
        report the issue with any details you can provide about the steps leading
        up to this error.


        You can submit your report here:
        https://github.com/jpjacobpadilla/Google-Colab-Selenium/issues/new

        We apologize for any inconvenience!
    

In [ ]:
driver = gs.Chrome()  # Ensure this is correctly configured for your setup
driver.get(url)
# Wait for the lw_cal_event_list to be rendered on the page
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME,"d3-l-wrap"))
    )
except Exception as e:
    print(f"Error: {e}")

soup = BeautifulSoup(driver.page_source, 'html.parser')

<IPython.core.display.Javascript object>

In [ ]:
table=soup.find('div', {'class': "sc-eCBGDA jsRppB"})

In [ ]:
Day=table.find_all('div', {'class': "sc-hPMVTF lixyng"})
content=table.find_all('div', {'class': "sc-bhvJaM cLBylL"})

In [ ]:
time[0]

'7'

In [ ]:
len(time)

11

In [ ]:
content[11].find('span',"sc-fqCdsd jJBPhl").get_text()

'7:00 PM EDT'

In [ ]:
events=[]
for i in range(11,len(Day)):
  date=Day[i].get_text()
  matchup=content[i].find("div","sc-cmSatC dbWQrc").get_text()
  time=content[i].find('span',"sc-fqCdsd jJBPhl").get_text()
  networks=content[i].find("div","sc-hhTLmD fEEwQB").get_text()
  event={
      'matchup':matchup,
      'time':date+"|"+time,
      'networks':networks,
      "location":"PPG Paints Arena, Pittsburgh, PA"
  }
  print(event)
  events.append(event)

{'matchup': 'Buffalo@Pittsburgh', 'time': 'Wednesday, October 16|7:00 PM EDT', 'networks': '', 'location': 'PPG Paints Arena, Pittsburgh, PA'}
{'matchup': 'Carolina@Pittsburgh', 'time': 'Friday, October 18|7:00 PM EDT', 'networks': 'SN-PIT, BSSO', 'location': 'PPG Paints Arena, Pittsburgh, PA'}
{'matchup': 'Pittsburgh@Winnipeg', 'time': 'Sunday, October 20|3:00 PM EDT', 'networks': 'SN-PIT', 'location': 'PPG Paints Arena, Pittsburgh, PA'}
{'matchup': 'Pittsburgh@Calgary', 'time': 'Tuesday, October 22|9:30 PM EDT', 'networks': 'SN-PIT', 'location': 'PPG Paints Arena, Pittsburgh, PA'}
{'matchup': 'Pittsburgh@Edmonton', 'time': 'Friday, October 25|9:00 PM EDT', 'networks': 'SN-PIT', 'location': 'PPG Paints Arena, Pittsburgh, PA'}
{'matchup': 'Pittsburgh@Vancouver', 'time': 'Saturday, October 26|10:00 PM EDT', 'networks': 'SN-PIT', 'location': 'PPG Paints Arena, Pittsburgh, PA'}
{'matchup': 'Minnesota@Pittsburgh', 'time': 'Tuesday, October 29|7:00 PM EDT', 'networks': 'SN-PIT, BSN, BSWI', 

In [ ]:
import requests
from bs4 import BeautifulSoup
# 发送HTTP请求并获取HTML内容
url = 'https://en.wikipedia.org/wiki/Pittsburgh_Penguins'
response = requests.get(url)
html = response.text
# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(html, 'html.parser')
# 提取所需数据
data = []
for tag in soup.find_all('div', {'class': 'mw-content-ltr'}):
  for p in tag.find_all('p'):
    text = p.get_text()
    if text:
        data.append(text)

In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/penguins.json', 'r') as f:
    ff = json.load(f)

In [ ]:

ff['wikipedia']=data


In [ ]:
with open('/content/drive/MyDrive/11711/assignment2/data/penguins.json', 'w') as f:
    json.dump(ff, f, indent=4)

In [ ]:
content

#general information

In [ ]:
import requests
from bs4 import BeautifulSoup
# 发送HTTP请求并获取HTML内容
url = 'https://www.visitpittsburgh.com/things-to-do/pittsburgh-sports-teams/'
response = requests.get(url)
html = response.text
# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(html, 'html.parser').get_text()


In [ ]:
soup.split('\n')

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Pittsburgh, PA Sports Teams | Steelers, Pirates, Penguins & Riverhounds | Visit Pittsburgh',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Skip to content',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Newsletter',
 '',
 '',
 'Discover Your Persona',
 '',
 '',
 '',
 '',
 '',
 '                    51°F',
 '                                    ',
 '',
 '',
 '0',
 'Saved',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Navigation',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' ',
 '',
 '',
 '',
 '',
 'Things to DoFree Things to DoFamily FunTours & SightseeingOutdoor AdventureArts & CultureShoppingHealth & WellnessPittsburgh Sports TeamsCasinos & GamingFilm FestivalsEvents & FestivalsSubmit Your EventThis Week in PittsburghAnnua

# museum

In [1]:
import re

In [2]:
with open('museum.txt', 'r') as f:
    text = f.read()
    text = text.split('\n')

In [12]:
for i in range(len(text)):
    print(repr(text[i]))

'American Jewish Museum'
'Andy Warhol Museum'
'August Wilson Center for African American Culture'
'Bicycle Heaven'
'Carnegie Museum of Art'
'Carnegie Museum of Natural History'
'Kamin Science Center'
'Center for PostNatural History'
"Children's Museum of Pittsburgh"
'The Clemente Museum'
'Elmer H. Grimm Sr. Pharmacy Museum'
'Fort Pitt Museum'
'The Frick Pittsburgh'
'Heinz History Center'
'Kelso Museum of Near Eastern Archaeology'
'Mattress Factory'
'Miller Gallery at Carnegie Mellon University'
'Miniature Railroad & Village'
'Nationality Rooms'
'Photo Antiquities Museum of Photographic History'
'Pittsburgh Center for the Arts'
'Pittsburgh Glass Center'
'Randyland'
'Silver Eye Center for Photography'
'Society for Contemporary Craft'
'Soldiers and Sailors Memorial Hall and Museum'
'Stephen Foster Memorial Museum'
'ToonSeum'
'Trundle Manor'
'University Arts Gallery'
'University of Pittsburgh Dental Museum'
'Western Pennsylvania Sports Museum at the History Center'
'Wood Street Galleries'


In [13]:
for i in range(len(text)):
  text[i]=text[i].replace(' ','_')
  print(text[i])

American_Jewish_Museum
Andy_Warhol_Museum
August_Wilson_Center_for_African_American_Culture
Bicycle_Heaven
Carnegie_Museum_of_Art
Carnegie_Museum_of_Natural_History
Kamin_Science_Center
Center_for_PostNatural_History
Children's_Museum_of_Pittsburgh
The_Clemente_Museum
Elmer_H._Grimm_Sr._Pharmacy_Museum
Fort_Pitt_Museum
The_Frick_Pittsburgh
Heinz_History_Center
Kelso_Museum_of_Near_Eastern_Archaeology
Mattress_Factory
Miller_Gallery_at_Carnegie_Mellon_University
Miniature_Railroad_&_Village
Nationality_Rooms
Photo_Antiquities_Museum_of_Photographic_History
Pittsburgh_Center_for_the_Arts
Pittsburgh_Glass_Center
Randyland
Silver_Eye_Center_for_Photography
Society_for_Contemporary_Craft
Soldiers_and_Sailors_Memorial_Hall_and_Museum
Stephen_Foster_Memorial_Museum
ToonSeum
Trundle_Manor
University_Arts_Gallery
University_of_Pittsburgh_Dental_Museum
Western_Pennsylvania_Sports_Museum_at_the_History_Center
Wood_Street_Galleries


In [31]:
names=['/content/drive/MyDrive/11711/assignment2/data/'+name+'.json' for name in text]
print(names)

['/content/drive/MyDrive/11711/assignment2/data/American_Jewish_Museum.json', '/content/drive/MyDrive/11711/assignment2/data/Andy_Warhol_Museum.json', '/content/drive/MyDrive/11711/assignment2/data/August_Wilson_Center_for_African_American_Culture.json', '/content/drive/MyDrive/11711/assignment2/data/Bicycle_Heaven.json', '/content/drive/MyDrive/11711/assignment2/data/Carnegie_Museum_of_Art.json', '/content/drive/MyDrive/11711/assignment2/data/Carnegie_Museum_of_Natural_History.json', '/content/drive/MyDrive/11711/assignment2/data/Kamin_Science_Center.json', '/content/drive/MyDrive/11711/assignment2/data/Center_for_PostNatural_History.json', "/content/drive/MyDrive/11711/assignment2/data/Children's_Museum_of_Pittsburgh.json", '/content/drive/MyDrive/11711/assignment2/data/The_Clemente_Museum.json', '/content/drive/MyDrive/11711/assignment2/data/Elmer_H._Grimm_Sr._Pharmacy_Museum.json', '/content/drive/MyDrive/11711/assignment2/data/Fort_Pitt_Museum.json', '/content/drive/MyDrive/11711/

In [18]:
links=['https://en.wikipedia.org/wiki/'+name for name in text]
print(links)
with open('museum_link.txt', 'w') as f:
    for link in links:
        f.write(link+'\n')

['https://en.wikipedia.org/wiki/American_Jewish_Museum', 'https://en.wikipedia.org/wiki/Andy_Warhol_Museum', 'https://en.wikipedia.org/wiki/August_Wilson_Center_for_African_American_Culture', 'https://en.wikipedia.org/wiki/Bicycle_Heaven', 'https://en.wikipedia.org/wiki/Carnegie_Museum_of_Art', 'https://en.wikipedia.org/wiki/Carnegie_Museum_of_Natural_History', 'https://en.wikipedia.org/wiki/Kamin_Science_Center', 'https://en.wikipedia.org/wiki/Center_for_PostNatural_History', "https://en.wikipedia.org/wiki/Children's_Museum_of_Pittsburgh", 'https://en.wikipedia.org/wiki/The_Clemente_Museum', 'https://en.wikipedia.org/wiki/Elmer_H._Grimm_Sr._Pharmacy_Museum', 'https://en.wikipedia.org/wiki/Fort_Pitt_Museum', 'https://en.wikipedia.org/wiki/The_Frick_Pittsburgh', 'https://en.wikipedia.org/wiki/Heinz_History_Center', 'https://en.wikipedia.org/wiki/Kelso_Museum_of_Near_Eastern_Archaeology', 'https://en.wikipedia.org/wiki/Mattress_Factory', 'https://en.wikipedia.org/wiki/Miller_Gallery_at_C

In [19]:
with open('museum_link.txt', 'r') as f:
    links = f.read().splitlines()
print(links)

['https://en.wikipedia.org/wiki/American_Jewish_Museum', 'https://en.wikipedia.org/wiki/The_Andy_Warhol_Museum', 'https://en.wikipedia.org/wiki/August_Wilson_African_American_Cultural_Center', 'https://en.wikipedia.org/wiki/Bicycle_Heaven', 'https://en.wikipedia.org/wiki/Carnegie_Museum_of_Art', 'https://en.wikipedia.org/wiki/Carnegie_Museum_of_Natural_History', 'https://en.wikipedia.org/wiki/Carnegie_Science_Center', 'https://en.wikipedia.org/wiki/Center_for_PostNatural_History', 'https://en.wikipedia.org/wiki/Children%27s_Museum_of_Pittsburgh', 'https://en.wikipedia.org/wiki/The_Clemente_Museum', 'https://en.wikipedia.org/wiki/Salk_Hall#Pharmacy_Museum', 'https://en.wikipedia.org/wiki/Fort_Pitt_Museum', 'https://en.wikipedia.org/wiki/The_Frick_Pittsburgh', 'https://en.wikipedia.org/wiki/Heinz_History_Center', 'https://en.wikipedia.org/wiki/Pittsburgh_Theological_Seminary#Kelso_Museum_of_Near_Eastern_Archaeology', 'https://en.wikipedia.org/wiki/Mattress_Factory', 'https://en.wikipedia

In [28]:
def build_file_name(url):
    return 'raw_documents/'+'-'.join(url.split("/")[2:]).replace('.','-') + ".json"
import requests
from bs4 import BeautifulSoup
import json

def scrape_wikipedia_to_json(url, output_file):

    # Send request to the URL
    response = requests.get(url)

    # Parse the page content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the correct div based on the updated structure
    content = soup.find('div', {'id': 'mw-content-text'})

    # Dictionary to store the result
    data = {}
    current_h1 = None
    current_h2 = None
    current_h3 = None
    current_h4 = None
    current_h5 = None
    current_h6 = None
    flag=False

    # Iterate over headings and paragraphs
    for tag in content.find_all(['h2', 'h3', 'h4', 'h5', 'h6', 'p']):
        if tag.name == 'h2':
            current_h2 = tag.get_text(strip=True)
            data[current_h2] = {}
            current_h3 = current_h4 = current_h5 = current_h6 = None
        elif tag.name == 'h3' and current_h2:
            current_h3 = tag.get_text(strip=True)
            data[current_h2][current_h3] = {}
            current_h4 = current_h5 = current_h6 = None
        elif tag.name == 'h4' and current_h3:
            current_h4 = tag.get_text(strip=True)
            data[current_h2][current_h3][current_h4] = {}
            current_h5 = current_h6 = None
        elif tag.name == 'h5' and current_h4:
            current_h5 = tag.get_text(strip=True)
            data[current_h2][current_h3][current_h4][current_h5] = {}
            current_h6 = None
        elif tag.name == 'h6' and current_h5:
            current_h6 = tag.get_text(strip=True)
            data[current_h2][current_h3][current_h4][current_h5][current_h6] = {}
        elif tag.name == 'p':
            if current_h6:
                data[current_h2][current_h3][current_h4][current_h5][current_h6].setdefault('content', []).append(tag.get_text(strip=True))
            elif current_h5:
                data[current_h2][current_h3][current_h4][current_h5].setdefault('content', []).append(tag.get_text(strip=True))
            elif current_h4:
                data[current_h2][current_h3][current_h4].setdefault('content', []).append(tag.get_text(strip=True))
            elif current_h3:
                data[current_h2][current_h3].setdefault('content', []).append(tag.get_text(strip=True))
            elif current_h2:
                data[current_h2].setdefault('content', []).append(tag.get_text(strip=True))
            elif flag==False:
                flag=True
                data['introduction']=tag.get_text(strip=True)

    # Save the dictionary as a JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"Content successfully written to {output_file}")


In [32]:
url_list = links
output_file_list = names
for url, output_file in zip(url_list, output_file_list):
    scrape_wikipedia_to_json(url, output_file)

Content successfully written to /content/drive/MyDrive/11711/assignment2/data/American_Jewish_Museum.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/Andy_Warhol_Museum.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/August_Wilson_Center_for_African_American_Culture.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/Bicycle_Heaven.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/Carnegie_Museum_of_Art.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/Carnegie_Museum_of_Natural_History.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/Kamin_Science_Center.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/Center_for_PostNatural_History.json
Content successfully written to /content/drive/MyDrive/11711/assignment2/data/Children's_Museum_of_Pittsburgh.json
Cont